    !pip install stopit

# Benchmark the reusability of literate coding documents.

In [1]:
    from collections import defaultdict
    from importnb import Notebook
    from pathlib import Path 
    from stopit import ThreadingTimeout
    import os, sys
    import pandas as pd, importlib
    from IPython import get_ipython
    from IPython.utils.capture import capture_output
    from tqdm import tqdm_notebook as progress
    from ricker import TqdmAudioTicker as progress
    import pandas
    from toolz.curried import *; from toolz.curried.operator import *; dunder="__%s__"
    from hypothesis import given, strategies

In [2]:
    projects = ("https://github.com/jakevdp/PythonDataScienceHandbook", 'notebooks'), ("https://github.com/AllenDowney/ThinkDSP", 'code')

In [3]:
    timeout = os.environ.get('CI', False) and 4 or .1

In [4]:
    def project_info(project): return project[0], os.sep.join((project[0].rsplit('/', 1)[1], project[1]))

In [5]:
    def load_notebooks_from_a(project):
        repo, project = project_info(project)
        global timeout
        current = os.getcwd()
        if not Path
        !git clone --depth 1 $repo
        name = Path(project).parent.name
        try:
            notebooks = list(Path(project).glob('*.ipynb'))
            bar = progress(notebooks)
            for notebook in bar:
                bar.set_description(notebook.name)
                with ThreadingTimeout(timeout), capture_output() as captured:
                    try: module = Notebook.load(notebook); module.__output__ = captured
                    except BaseException as e: 
                        module = importlib.util.module_from_spec(importlib.machinery.ModuleSpec(Path(notebook).stem, str(notebook), origin=str(notebook)))
                        with importlib._bootstrap._installed_safely(module): 
                            module.__file__ = str(notebook)
                            module.__exception__ = e
                os.chdir(current)
        except KeyboardInterrupt:...
        finally: os.chdir(current)

    _load_notebooks_from_a = given(project=strategies.one_of(*map(strategies.just, projects)))(load_notebooks_from_a)

In [6]:
    __name__ == '__main__' and _load_notebooks_from_a()

fatal: destination path 'PythonDataScienceHandbook' already exists and is not an empty directory.


hypothesis_temporary_module_608bfba9cbbbde8998bde8bccbfb1c8112e27cee:2: HypothesisDeprecationWarning: Test took 10360.00ms to run. In future the default deadline setting will be 200ms, which will make this an error. You can set deadline to an explicit value of e.g. 10400 to turn tests slower than this into an error, or you can set it to None to disable this check entirely.
fatal: destination path 'ThinkDSP' already exists and is not an empty directory.


Turn the contents of existing projects into dataframes based on a project directories.

In [25]:
    def project_frames(*projects):
        s = pd.Series(sys.modules)
        all_the_files = sum((list(Path(object).rglob('*.ipynb')) for repo, object in map(project_info, projects)), [])
        return s[s.apply(
            lambda x: Path(getattr(x, '__file__', '') or '') in all_the_files
        ).dropna()].apply(compose(pd.Series, vars))

In [27]:
    def _extract_dataframes_from_sys_modules():
        assert len(project_frames(*projects))